# Clean Operator Data Reports
Code author: Sahar H. El Abbadi
Date started: 2022-02-22
Date last edited: 2022-03-23

### Notes
For Carbon Mapper, GHGSat, Kairos, and Methane Air data reports:
- This notebook loads the raw excel file submitted by the team. It applies a team-specific function to clean the report. Output is a standardized report that will be used to generate overpass summaries in 03_summarize_results.
- All raw report inputs are saved in 00_raw_data. No changes are manually made to operator reports. All cleaning is handled in Python.


In [1]:
# Imports
import pandas as pd
import pathlib
from methods_clean_reports import clean_cm, clean_ghgsat, clean_kairos


# Load and clean raw data submitted by operators
### Notes on formatting:
- Operators added their own QC indicators, thus not all columns are uniform across reports
- Values left in the Excel file are replaced during import into PyCharm with "nan"
- Naming convention for dataframes: operator_stage

## Notes on Cleaning Operator Data

### Generate data frame with the following columns:
- Operator: name of operator (Kairos, GHGSat, CarbonMapper, scientificav)
- Stage: stage of unblinding (1, 2, or 3)
- PerformerExperimentID: Overpass number for a specific operator, incrementing by 1 for each overpass
- DateOfSurvey: date in YYYY-MM-DD format
- TimestampUTC: timestamp in UTC using 24 hour time
- QuantifiedPlume: boolean input, 1 indicates operator submitted a valid quantification estimate for this overpass (excludes quantification estimates that are provided but fail operator QC standards)
- FacilityEmissionRate: estimated emissions in kgh
- FacilityEmissionRateUpper: upper bound of uncertainty on quantification estimate
- FacilityEmissionRateLower: lower bound of uncertainty on quantification estimate
- UncertaintyType: type of uncertainty for upper and lower values reported above
- OperatorWindspeed: operator reported windspeed in m/s
- QCFlag: operator specific QC flag, first digits indicate operator.

### Carbon Mapper: Import and Clean Stage 1 and 2 data

#### Submission details
- Stage 1 submitted on 2023-01-03
- Stage 2 submitted on 2023-02-13
- Stage 3 submitted 2023-02-28

#### QC Indicator:
Column: "Good Quality (Y/N)"
- Y = good quality, quantification included for this stage
- N = not good quality, quantification estimate included for potential use in a later stage, but not included in this stage
- nan = left blank by Carbon Mapper. All entries that are left blank correspond to overpasses in which CarbonMapper did not observe a released plume. For these cases, "Quantified" column reads "no_detect"


In [2]:
# Load Carbon Mapper Stage 1
cm_1_path = pathlib.PurePath('00_raw_reports', 'CM_Stage1_submitted-2023-01-03.xlsx')
cm_1 = pd.read_excel(cm_1_path, sheet_name='Survey Summary')

# Load Carbon Mapper Stage 2
cm_2_path = pathlib.PurePath('00_raw_reports', 'CM_Stage2_submitted-2023-02-13.xlsx')
cm_2 = pd.read_excel(cm_2_path, sheet_name='Survey Summary')

# Load Carbon Mapper Stage 3
cm_3_path = pathlib.PurePath('00_raw_reports', 'CM_Stage3_submitted-2023-02-28.xlsx')
cm_3 = pd.read_excel(cm_3_path, sheet_name='Survey Summary')


### Clean Carbon Mapper Data

#### Required data cleaning:

- QuantifiedPlume:
    - CarbonMapper reports whether they quantified a plume using two metrics:
        - "CR plume present (Y/N)" indicates whether they detected a plume. "N" indicates that they do not detect a plume, ie they are estimating 0 kgh
        - "Good Quality (Y/N)" indicates if they quantified the plume observed or not.
    - For data cleaning, if "CR plume present (Y/N)" == "Y" AND "Good Quality (Y/N)" == Y, then QuantifiedPlume = 1;
    - For now, I will exclude zero values from the quantification plot

- TimestampUTC: CarbonMapper reports in local time (UTC - 7), needs to be adjusted to UTC
- QCFlag: CarbonMapper only uses on QC indicator ("Good Quality (Y/N)"), so I use a QC flag of "CM-1" for all items that fail to pass their QC test. For items that pass QC test, I use "clear"

#### Notes:
- For detection capabilities, use "CR plume present (Y/N)" column. "Y" indicates detection by Carbon Mapper, "N" indicates no detection by Carbon Mapper

In [3]:
# Carbon Mapper data cleaning
# Carbon Mapper conducted 121 overpasses
total_overpass = 121

# Clean Stage 1 data
stage = 1
cm_1_clean = clean_cm(cm_report=cm_1, cm_overpasses=total_overpass, cm_stage=stage)

# Clean Stage 2 data
stage = 2
cm_2_clean = clean_cm(cm_report=cm_2, cm_overpasses=total_overpass, cm_stage=stage)

# Clean stage 3
stage = 3
cm_3_clean = clean_cm(cm_report=cm_3, cm_overpasses=total_overpass, cm_stage=stage)

# Save data
cm_1_clean.to_csv(pathlib.PurePath('01_clean_reports', 'cm_1_clean.csv'))
cm_2_clean.to_csv(pathlib.PurePath('01_clean_reports', 'cm_2_clean.csv'))
cm_3_clean.to_csv(pathlib.PurePath('01_clean_reports', 'cm_3_clean.csv'))


### GHGSAT Stage 1 and 2 data

#### Submission details:
- Stage 1 data submitted on 2022-11-21
- Stage 2 data submitted on 2022-12-23
- Stage 3 data same as Stage 2, submitted 2023-02-17

#### QC Indicator:

Column: "QC Flag"
- 1 = Good conditions
- 2 = Emissions detected and quantified, but suboptimal conditions may affect SR
- 3 = Emissions detected, but not quantified due to suboptimal conditions
- 4 = Diffuse emission visible over site (presumably from previous release, due to low wind)
- 5 = Discarded (Bad weather/conditions, including clouds, cloud shadow, highly irregular aircraft trajectory, etc.)

#### Data processing notes
Unable to open the .xlsx file provided by GHGSat in Python, possibly related to read-only restrictions. I have saved the relevant data sheets as csv files to load instead, original submissions by GHGSat are included in 00_raw_data as .xlsx files.

In [2]:
# GHGSat Stage 1
ghg_1_path = pathlib.PurePath('00_raw_reports', 'GHG_Stage1_submitted-2022-11-21.csv')
ghg_1 = pd.read_csv(ghg_1_path)

# GHGSat Stage 2
ghg_2_path = pathlib.PurePath('00_raw_reports', 'GHG_Stage2_submitted-2022-12-23.csv')
ghg_2 = pd.read_csv(ghg_2_path)

### Clean GHGSat Data

#### Required Data Cleaning:
- PerformerExperimentID:
    - Current ExperimentID is a 54 character string including indicators for date, time, line number and frame. The last entry is an indicator for overpass number that starts at 1 and incrementally increases with each overpass
    - Replace this with GH-1, GH-2, etc.
- TimestampUTC: GHGSat reports in local time (UTC - 7), needs to be adjusted to UTC
- QCFlag: add "GH-" prefix to the QC flags used by GHGSat
- QuantifiedPlume: if QCFlag (defined below in notes) is 1 or 2, I will consider this a valid and submitted quantification.
- UncertaintyType: 1-sigma

#### Notes:
- QC Flags as defined by GHGSat:
    - 1 = "Good conditions"
    - 2 = "Emissions detected and quantified, but suboptimal conditions may affect SR" (what is SR?)
    - 3 = "Emissions detected, but not quantified due to suboptimal conditions"
    - 4 = "Diffuse emissions visible over site (presumably from previous release, due to low wind)"
    - 5 = "Discarded (Bad weather/conditions, including clouds, cloud shadow, highly irregular aircraft trajectory, etc.)"
- It looks like in some instances, GHGSat quantified plumes with QC Flag = 4, and in some places they did not. Will need to look into this more rigorously
- QuantifiedPlume: compare QCFlags 1 and 2 for any significant difference in error estimation
- Manually correct incorrect timestamp for Overpass ID #75: per email correspondance with Marianne Girard adn March 22, 2023 the timestamp should be 2022-11-02 16:58:28 UTC

In [3]:
# GHGSat Data Cleaning

# GHGSat conducted 192 overpasses
ghg_overpass = 192

# Import Stage 1 data
ghg_stage = 1
ghg_1_clean = clean_ghgsat(ghg_report=ghg_1, ghg_overpasses=ghg_overpass, ghg_stage=ghg_stage)

# Import Stage 2 data
ghg_stage = 2
ghg_2_clean = clean_ghgsat(ghg_report=ghg_2, ghg_overpasses=ghg_overpass, ghg_stage=ghg_stage)

# Save data
ghg_1_clean.to_csv(pathlib.PurePath('01_clean_reports', 'ghg_1_clean.csv'))
ghg_2_clean.to_csv(pathlib.PurePath('01_clean_reports', 'ghg_2_clean.csv'))




### Kairos Stage data: individual pods (LS23 and LS25)

#### Submission details
- Stage 1 submitted on 2022-11-17
- Stage 2 submitted on 2022-12-20
- Stage 3 submitted on 2023-02-23
- Kairos submitted data for two pods, LS23 and LS25. They analyzed the data independently, but did not report this until after testing was complete.

#### QC Indicator:
(I ran the UNIQUE function in Excel to identify values in their original report)
- "Plane deviated from flightline"
- "PARTIAL DETECTION"
- "Cutoff - low confidence quantification"
- "Excessive methane pooling near site"
- "Excessive methane pooling over site" (appears twice - possible extra space at end?)
- "Plane deviation from flightpath"
- "Glare"

#### Data processing notes
Unable to open the .xlsx file provided by GHGSat in Python, possibly related to read-only restrictions. I have saved the relevant data sheets as csv files to load instead, original submissions by GHGSat are included in 00_raw_data as .xlsx files.


In [6]:
# Kairos Stage 1
kairos_ls23_1_path = pathlib.PurePath('00_raw_reports', 'Kairos_Stage1_podLS23_submitted-2022-11-17.csv')
kairos_ls25_1_path = pathlib.PurePath('00_raw_reports', 'Kairos_Stage1_podLS25_submitted-2022-11-17.csv')
kairos_ls23_1 = pd.read_csv(kairos_ls23_1_path)
kairos_ls25_1 = pd.read_csv(kairos_ls25_1_path)

# Kairos Stage 2
kairos_ls23_2_path = pathlib.PurePath('00_raw_reports', 'Kairos_Stage2_podLS23_submitted-2022-12-20.csv')
kairos_ls25_2_path = pathlib.PurePath('00_raw_reports', 'Kairos_Stage2_podLS25_submitted-2022-12-20.csv')
kairos_ls23_2 = pd.read_csv(kairos_ls23_2_path)
kairos_ls25_2 = pd.read_csv(kairos_ls25_2_path)

# Kairos Stage 3
kairos_ls23_3_path = pathlib.PurePath('00_raw_reports', 'Kairos_Stage3_podLS23_submitted-2023-02-23.csv')
kairos_ls25_3_path = pathlib.PurePath('00_raw_reports', 'Kairos_Stage3_podLS25_submitted-2023-02-23.csv')
kairos_ls23_3 = pd.read_csv(kairos_ls23_3_path)
kairos_ls25_3 = pd.read_csv(kairos_ls25_3_path)

### Clean Kairos Data: reports for individual pods

#### Required Data Cleaning:
QC Flag:
- Kairos uses the following QC flags in their column "Kairos Flag for Dropped Passes or Uncertain Rate Quantification":
    - "Plane deviated from flightline"
    - "PARTIAL DETECTION"
    - "Cutoff - low confidence quantification"
    - "Excessive methane pooling near site"
    - "Excessive methane pooling over site" (appears twice - possible extra space at end?)
    - "Plane deviation from flightpath"
    - "Glare"
- Kairos reported for pods LS23 and LS25

#### Notes:
- First, clean reports for each pod as reported by Kairos.
- Next, generate a combined Kairos clean dataframe with FacilityEmissionRate equal to the average of the FacilityEmissionRate reported by both LS23 and LS25. The individual LS23 and LS25 values will be the upper and lower uncertainty


In [7]:
# Kairos Data Cleaning of LS23 and LS25 reports

# Kairos conducted 349 overpasses
kairos_overpass = 349

# Stage 1, pods LS23 and LS25
kairos_stage = 1
kairos_1_ls23_clean = clean_kairos(kairos_report=kairos_ls23_1, kairos_overpasses=kairos_overpass, kairos_stage=kairos_stage)
kairos_1_ls25_clean = clean_kairos(kairos_report=kairos_ls25_1, kairos_overpasses=kairos_overpass, kairos_stage=kairos_stage)

# Stage 2, pods LS23 and LS25
kairos_stage = 2
kairos_2_ls23_clean = clean_kairos(kairos_report=kairos_ls23_2, kairos_overpasses=kairos_overpass, kairos_stage=kairos_stage)
kairos_2_ls25_clean = clean_kairos(kairos_report=kairos_ls25_2, kairos_overpasses=kairos_overpass, kairos_stage=kairos_stage)

# Stage 3, pods LS23 and LS25
kairos_stage = 3
kairos_3_ls23_clean = clean_kairos(kairos_report=kairos_ls23_3, kairos_overpasses=kairos_overpass, kairos_stage=kairos_stage)
kairos_3_ls25_clean = clean_kairos(kairos_report=kairos_ls25_3, kairos_overpasses=kairos_overpass, kairos_stage=kairos_stage)

# Save Data
kairos_1_ls23_clean.to_csv(pathlib.PurePath('01_clean_reports', 'kairos_1_ls23_clean.csv'))
kairos_1_ls25_clean.to_csv(pathlib.PurePath('01_clean_reports', 'kairos_1_ls25_clean.csv'))
kairos_2_ls23_clean.to_csv(pathlib.PurePath('01_clean_reports', 'kairos_2_ls23_clean.csv'))
kairos_2_ls25_clean.to_csv(pathlib.PurePath('01_clean_reports', 'kairos_2_ls25_clean.csv'))
kairos_3_ls23_clean.to_csv(pathlib.PurePath('01_clean_reports', 'kairos_3_ls23_clean.csv'))
kairos_3_ls25_clean.to_csv(pathlib.PurePath('01_clean_reports', 'kairos_3_ls25_clean.csv'))

#### Generate combined notebook
- Because Kairos reported results as individual pods, we combine these results in the main manuscript. This script generates the combined clean report.
- Combined report averages the values found by both pods for FacilityEmissionRate
- FacilityEmissionRateUpper and FacilityEmissionRateLower are set to the individual values of the pods themselves
- If one pod detects a release and the other does not, we use the value of the pod that detected the release
- Similarly, if one pod experiences a QC issue but the other does not, we use the value by the pod that reports values (ie did not have a QC problem).

In [8]:
import pandas as pd
import pathlib
from methods_clean_reports import make_kairos_combo

kairos_overpass = 349
kairos_stage = 1
kairos_1_clean = make_kairos_combo(kairos_overpass, kairos_stage)

kairos_stage = 2
kairos_2_clean = make_kairos_combo(kairos_overpass, kairos_stage)

kairos_stage = 3
kairos_3_clean = make_kairos_combo(kairos_overpass, kairos_stage)

### Scientific Aviation Data

#### Submission details
- Only submitted Phase I estimates
- Submitted on 2023-01-21

#### QC Indicator
Initial Spreadsheet reports quantification estimates for all releases, and indicates if release does not meet Scientific Aviation criteria by printing the text in grey. All greyed out releases include a comment explaining why the measurement was not valid

- Entries in the "Comments" column:
- "*not enough of plumes captured at low end due to restrictions in altitude (powerlines)"
- "* same as above" (referring to powerline problem)
- "*too few laps"
- "*too few laps; upwind interference from landfill"
- "*not enough of plume captured near surface"

#### Generating clean report
Scientific Aviation did not submit a report using the Stanford provided template. Because formatting of the excel file is not readily machine-readable, Sahar El Abbadi manual generated the clean version of the data. Because Scientific Aviation uses a different measurement approach, the following columns are used in generating the clean dataframe:
- Operator
- Stage
- overpass_id
- DateOfSurvey
- StartUTC
- EndUTC
- Detected
- QuantifiedPlume
- FacilityEmissionRate
- FacilityEmissionRateUpper
- FacilityEmissionRateLower
- UncertaintyType
- OperatorWindspeed
- QCFlag
- OperatorKeep

In [9]:
# Scientific Aviation data import - this was generated manually by Sahar because SciAV did not submit using report template

sciav_path = pathlib.PurePath('00_raw_reports', 'SciAv_Stage1_submitted-2023-02-21.xlsx')
sciav_1 = pd.read_excel(sciav_path)

sciav_clean_path = pathlib.PurePath('01_clean_reports', 'sciav_1_clean.csv')
sciav_1_clean = pd.read_csv(sciav_clean_path)

## Methane Air data cleaning

Notes :
- MAIR submitted an extra column with plume area, column title is PlumeLength(m)(Area)^.5
- Uncertainty type: they report 95% confidence interval and "MINIMUM" -> need to ask for clarification on this


In [4]:
# Methane Air data report
# Imports
import pandas as pd
import pathlib
from methods_clean_reports import clean_mair

# load raw file
mair_path = pathlib.PurePath('00_raw_reports', 'MAIR_Stage1_submitted_2023-03-22.csv')
mair_1 = pd.read_csv(mair_path)

mair_report = mair_1
mair_overpasses = 24
mair_stage = 1
mair_1_clean = clean_mair(mair_report, mair_overpasses, mair_stage)

# Save Data
mair_1_clean.to_csv(pathlib.PurePath('01_clean_reports', 'mair_1_clean.csv'))


### Generate Methane Air individual results report
Methane Air submitted a report which averaged their two approaches:  mIME and DI methods. On March 31st, they submitted results for each individual method.

They did not use the data reporting template, make and save a copy here using their submission on 2023-03-22

In [9]:
import pathlib
import pandas as pd

# load raw file from 03/22/2023
mair_path = pathlib.PurePath('00_raw_reports', 'MAIR_Stage1_submitted_2023-03-22.csv')
mair_raw = pd.read_csv(mair_path)

# load mIME results from csv file
mair_path_raw = pathlib.PurePath('00_raw_reports', 'MAIR_2023-03-31_SubmissionDocs', 'outputs_and_raw_mIME_DI_20230327.csv')

# make mIME method
mair_raw_df = pd.read_csv(mair_path_raw)
mair_mIME = mair_raw.copy()
mair_mIME['FacilityEmissionRate'] = mair_raw_df['mIME_flux_kg.hr']
mair_mIME['FacilityEmissionRateUpper'] = mair_raw_df['mIME_upper']
mair_mIME['FacilityEmissionRateLower'] = mair_raw_df['mIME_lower']

# make DI method
mair_DI = mair_raw.copy()
mair_DI['FacilityEmissionRate'] = mair_raw_df['DI_flux_kg.hr']
mair_DI['FacilityEmissionRateUpper'] = mair_raw_df['DI_upper']
mair_DI['FacilityEmissionRateLower'] = mair_raw_df['DI_lower']


mair_mIME_1_clean = clean_mair(mair_report=mair_mIME, mair_overpasses=24, mair_stage=1)
mair_DI_1_clean = clean_mair(mair_report=mair_DI, mair_overpasses=24, mair_stage=1)

mair_mIME_1_clean.to_csv(pathlib.PurePath('01_clean_reports', 'mair_mIME_1_clean.csv'))
mair_DI_1_clean.to_csv(pathlib.PurePath('01_clean_reports', 'mair_DI_1_clean.csv'))
